In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [14]:
class AlzNet(nn.Module):
    def __init__(self, p=0.2):
        super().__init__()
        self.conv1 = nn.Conv2d(400, 528, 9)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(96, 96, 7)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(45, 45, 5)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Conv2d(20, 20, 5)
        self.pool4 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(8, 8, 3)
        self.pool5 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(288, 121)
        self.dropout = nn.Dropout(p)
        self.fc2 = nn.Linear(121, 1)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = self.pool4(F.relu(self.conv4(x)))
        x = self.pool5(F.relu(self.conv5(x)))
        x = torch.flatten(x)
        x = self.dropout(F.relu(self.fc1(x)))
        x = F.sigmoid(self.fc2(x))
        return x


In [6]:
# batch_size = 4

# trainset = []
# testset = []

# trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,shuffle=True, num_workers=2)
# testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,shuffle=False, num_workers=2)

# classes = ('alzheimer', 'control')

In [8]:
import numpy as np

data = np.array([])
labels = np.array([])

with open('data.npy', 'rb') as f:
    data = np.load(f)

with open('labels.npy', 'rb') as f:
    labels = np.load(f)

In [16]:
alznet = AlzNet()

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alznet.parameters(), lr=0.001, momentum=0.9)

In [18]:
epochs = 5

for epoch in range(epochs):
    running_loss = 0.0
    for i, image in enumerate(data):
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = alznet(image)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

TypeError: conv2d() received an invalid combination of arguments - got (numpy.ndarray, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !Parameter!, !Parameter!, !tuple!, !tuple!, !tuple!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !Parameter!, !Parameter!, !tuple!, !tuple!, !tuple!, int)
